### Import dependencies

In [1]:
import pandas as pd
import numpy as np
from functools import reduce
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time

### Merge datasets

In [2]:
# Read in data
cpi = pd.read_csv("resources/cpi_final.csv")
gdp = pd.read_csv("resources/gdp_final.csv")
gdp_pct = pd.read_csv("resources/gdp_pct_chg_final.csv")
houst = pd.read_csv("resources/housing_starts_final.csv")
opg = pd.read_csv("resources/output_gap_final.csv")
rec_dt = pd.read_csv("resources/recession_dates_final.csv")
unrate = pd.read_csv("resources/unemployment_rate_final.csv")
fed_funds = pd.read_csv("resources/fed_funds_final.csv")
yield10_2 = pd.read_csv("resources/10YT_minus_2YT_final.csv")
fred = pd.read_csv("resources/FRED_data.csv")

In [3]:
# Combine all data sets into one data frame
dfs = [cpi, gdp, gdp_pct, houst, opg, rec_dt, unrate, fed_funds, yield10_2, fred]
df = reduce(lambda left,right: pd.merge(left,right,on=['quarter'],how='outer'), dfs)
df.head()

,quarter,avg_consumer_price_index,date_x,gdp,date_y,gdp_pct_change,avg_housing_starts,date_x,output_gap,date_y,...,10YT_minus_2YT_percent_change_prev_quarter,real_disp_pers_inc,personal_consumption_exp_excl_food_energy,cpi_US_total,tot_public_debt_as_pct_of_gdp,gross_private_domestic_invest,M2_velocity,median_sls_price_houses_sold_US,nat_rate_of_unemp_long_term,personal_consumption_expenditures
0,1947Q1,21.700000,1947-01-01,243.164,NaN,NaN,NaN,NaN,NaN,1947-01-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1947Q2,22.010000,1947-04-01,245.968,1947-04-01,4.7,NaN,NaN,NaN,1947-04-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1947Q3,22.490000,1947-07-01,249.585,1947-07-01,6.0,NaN,NaN,NaN,1947-07-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1947Q4,23.126667,1947-10-01,259.745,1947-10-01,17.3,NaN,NaN,NaN,1947-10-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1948Q1,23.616667,1948-01-01,265.742,1948-01-01,9.6,NaN,NaN,NaN,1948-01-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Drop date columns
df = df.drop(columns=['date_x','date_y'])

In [5]:
# Sort data frame by quarter
df = df.sort_values(by=['quarter'])

In [6]:
# Check dataset before removing nulls
df.tail()

,quarter,avg_consumer_price_index,gdp,gdp_pct_change,avg_housing_starts,output_gap,target,avg_unemployment_rate,fed_funds_avg_rate,fed_funds_percent_change_prev_quarter,...,10YT_minus_2YT_percent_change_prev_quarter,real_disp_pers_inc,personal_consumption_exp_excl_food_energy,cpi_US_total,tot_public_debt_as_pct_of_gdp,gross_private_domestic_invest,M2_velocity,median_sls_price_houses_sold_US,nat_rate_of_unemp_long_term,personal_consumption_expenditures
287,2018Q4,252.759000,20897.804,2.9,1185.000000,0.592021,0.0,3.566667,2.217097,0.152641,...,-0.078947,2.8,1.9,2.203131,105.15026,3725.234,1.462,322800.0,4.582,14211.920
288,2019Q1,253.311333,21098.827,3.9,1213.000000,0.848147,0.0,4.133333,2.401311,0.083088,...,-0.271429,4.5,1.6,1.644936,104.40334,3783.364,1.458,313000.0,4.577,14266.250
289,2019Q2,255.139333,21340.267,4.7,1255.666667,0.828815,NaN,3.500000,2.397813,-0.001457,...,0.254902,2.4,1.6,1.811376,103.20060,3749.471,1.457,322500.0,4.572,14511.176
290,2019Q3,256.273000,NaN,NaN,1282.000000,NaN,NaN,3.700000,2.197813,-0.083409,...,-0.484375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
311,2019Q4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.845625,-0.160245,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Drop rows with missing values
df = df.dropna()

In [8]:
# Check dataset after removing nulls
df.tail()

,quarter,avg_consumer_price_index,gdp,gdp_pct_change,avg_housing_starts,output_gap,target,avg_unemployment_rate,fed_funds_avg_rate,fed_funds_percent_change_prev_quarter,...,10YT_minus_2YT_percent_change_prev_quarter,real_disp_pers_inc,personal_consumption_exp_excl_food_energy,cpi_US_total,tot_public_debt_as_pct_of_gdp,gross_private_domestic_invest,M2_velocity,median_sls_price_houses_sold_US,nat_rate_of_unemp_long_term,personal_consumption_expenditures
284,2018Q1,249.250333,20163.159,5.0,1320.666667,0.202456,0.0,4.333333,1.448966,0.204683,...,-0.113861,6.9,1.8,2.214194,104.59493,3542.412,1.451,331800.0,4.597,13728.357
285,2018Q2,250.578667,20510.177,7.1,1259.666667,0.589182,0.0,3.833333,1.727176,0.192007,...,-0.251397,2.7,2.0,2.711887,103.33928,3561.592,1.461,315600.0,4.592,13939.828
286,2018Q3,251.828667,20749.752,4.8,1233.000000,0.821959,0.0,3.866667,1.923492,0.113663,...,-0.432836,3.3,2.0,2.640940,103.69309,3683.981,1.462,330900.0,4.587,14114.559
287,2018Q4,252.759000,20897.804,2.9,1185.000000,0.592021,0.0,3.566667,2.217097,0.152641,...,-0.078947,2.8,1.9,2.203131,105.15026,3725.234,1.462,322800.0,4.582,14211.920
288,2019Q1,253.311333,21098.827,3.9,1213.000000,0.848147,0.0,4.133333,2.401311,0.083088,...,-0.271429,4.5,1.6,1.644936,104.40334,3783.364,1.458,313000.0,4.577,14266.250


In [9]:
# Set index to quarter
df = df.set_index('quarter')

In [10]:
# Rename target column
df = df.rename(columns={'target':'recession_actual'})
df.head()

,avg_consumer_price_index,gdp,gdp_pct_change,avg_housing_starts,output_gap,recession_actual,avg_unemployment_rate,fed_funds_avg_rate,fed_funds_percent_change_prev_quarter,fed_funds_st_dev_rate,...,10YT_minus_2YT_percent_change_prev_quarter,real_disp_pers_inc,personal_consumption_exp_excl_food_energy,cpi_US_total,tot_public_debt_as_pct_of_gdp,gross_private_domestic_invest,M2_velocity,median_sls_price_houses_sold_US,nat_rate_of_unemp_long_term,personal_consumption_expenditures
quarter,,,,,,,,,,,,,,,,,,,,,
1976Q3,57.300000,1886.558,7.6,1557.000000,-2.199151,0.0,7.600000,5.283478,0.016956,0.100618,...,0.370833,3.2,6.0,5.518087,33.64333,328.307,1.717,44400.0,6.217,1158.806
1976Q4,58.133333,1934.273,10.5,1691.333333,-2.246705,0.0,7.333333,4.874239,-0.077456,0.211941,...,0.337386,2.6,6.0,5.069403,33.78753,337.650,1.699,45500.0,6.223,1192.408
1977Q1,59.200000,1988.648,11.7,1844.333333,-1.877175,0.0,8.233333,4.660667,-0.043817,0.148254,...,-0.095455,0.9,6.2,5.857741,33.65136,360.313,1.689,46300.0,6.227,1228.212
1977Q2,60.233333,2055.909,14.2,1918.666667,-0.776696,0.0,6.933333,5.157473,0.106595,0.332835,...,-0.052764,3.8,6.5,6.847698,32.80422,389.703,1.701,48900.0,6.232,1255.980
1977Q3,61.066667,2118.473,12.7,2009.000000,0.186001,0.0,6.800000,5.816413,0.127764,0.344309,...,-0.342175,5.7,6.6,6.682162,32.98791,414.134,1.713,48800.0,6.235,1286.905


### Shift data with sliding window technique

In [11]:
df['recession_1q_out'] = df['recession_actual'].shift(-1)
df['recession_2q_out'] = df['recession_actual'].shift(-2)
df['recession_4q_out'] = df['recession_actual'].shift(-4)

In [12]:
# Create three datasets -- 1 for each model (recession 1Qtr out, 2Qtrs out, 4Qtrs out)
df_q1 = df.drop(columns=['recession_2q_out','recession_4q_out','recession_actual'])
df_q2 = df.drop(columns=['recession_4q_out','recession_1q_out','recession_actual'])
df_q4 = df.drop(columns=['recession_1q_out','recession_2q_out','recession_actual'])

In [13]:
# Delete missing values
df_q1 = df_q1.dropna()
df_q2 = df_q2.dropna()
df_q4 = df_q4.dropna()
df_q4.tail()

,avg_consumer_price_index,gdp,gdp_pct_change,avg_housing_starts,output_gap,avg_unemployment_rate,fed_funds_avg_rate,fed_funds_percent_change_prev_quarter,fed_funds_st_dev_rate,10YT_minus_2YT_avg,...,real_disp_pers_inc,personal_consumption_exp_excl_food_energy,cpi_US_total,tot_public_debt_as_pct_of_gdp,gross_private_domestic_invest,M2_velocity,median_sls_price_houses_sold_US,nat_rate_of_unemp_long_term,personal_consumption_expenditures,recession_4q_out
quarter,,,,,,,,,,,,,,,,,,,,,
2017Q1,243.830000,19190.431,4.2,1230.666667,-0.861917,4.866667,0.698889,0.559490,0.098830,1.203333,...,4.9,1.8,2.539321,103.41831,3288.229,1.438,313100.0,4.616,13104.419,0.0
2017Q2,244.065000,19356.649,3.5,1169.333333,-0.752038,4.233333,0.947363,0.355527,0.098588,0.970000,...,2.7,1.6,1.901991,102.52061,3334.971,1.432,318200.0,4.611,13212.501,0.0
2017Q3,245.368333,19611.704,5.4,1175.333333,-0.396155,4.400000,1.153696,0.217797,0.024029,0.880000,...,2.3,1.5,1.966925,103.22866,3401.815,1.435,320500.0,4.607,13345.053,0.0
2017Q4,247.273333,19918.910,6.4,1259.666667,0.033653,3.900000,1.202778,0.042543,0.099120,0.673333,...,3.7,1.7,2.117557,102.88087,3457.732,1.442,337900.0,4.602,13586.267,0.0
2018Q1,249.250333,20163.159,5.0,1320.666667,0.202456,4.333333,1.448966,0.204683,0.083902,0.596667,...,6.9,1.8,2.214194,104.59493,3542.412,1.451,331800.0,4.597,13728.357,0.0


In [14]:
# Define y variables
y1 = df_q1['recession_1q_out']
y2 = df_q2['recession_2q_out']
y3 = df_q4['recession_4q_out']

In [15]:
# Drop target
df_q1 = df_q1.drop(columns=['recession_1q_out'])
df_q2 = df_q2.drop(columns=['recession_2q_out'])
df_q4 = df_q4.drop(columns=['recession_4q_out'])

In [16]:
# Define X
X_q1 = df_q1
X_q2 = df_q2
X_q4 = df_q4

### Split and scale data

In [17]:
# Split data into training and testing
X1_train, X1_test, y1_train, y1_test=train_test_split(X_q1, y1, train_size=0.8, random_state=42, stratify=y1)
X2_train, X2_test, y2_train, y2_test=train_test_split(X_q2, y2, train_size=0.8, random_state=42, stratify=y2)
X3_train, X3_test, y3_train, y3_test=train_test_split(X_q4, y3, train_size=0.8, random_state=42, stratify=y3)

In [19]:
# Create scaler object
X1_scaler = StandardScaler().fit(X1_train)
X2_scaler = StandardScaler().fit(X2_train)
X3_scaler = StandardScaler().fit(X3_train)

# X full scaler
X1_full_scaler = StandardScaler().fit(X_q1)
X2_full_scaler = StandardScaler().fit(X_q2)
X3_full_scaler = StandardScaler().fit(X_q4)

In [20]:
# Scale training data
X1_train_scaled = X1_scaler.transform(X1_train)
X2_train_scaled = X2_scaler.transform(X2_train)
X3_train_scaled = X3_scaler.transform(X3_train)

# Scale testing data
X1_test_scaled = X1_scaler.transform(X1_test)
X2_test_scaled = X2_scaler.transform(X2_test)
X3_test_scaled = X3_scaler.transform(X3_test)

#X1_full_scaled
X1_full_scaled = X1_full_scaler.transform(X_q1)
X2_full_scaled = X2_full_scaler.transform(X_q2)
X3_full_scaled = X3_full_scaler.transform(X_q4)

### Reshape data to fit LSTM format

In [21]:
# Method to reshape data
def reshape_data(obj):
    reshaped_obj = np.reshape(obj, (obj.shape[0], obj.shape[1], 1))
    return reshaped_obj

In [22]:
# Reshape training data
reshaped_X1_train_scaled = reshape_data(X1_train_scaled)
reshaped_X2_train_scaled = reshape_data(X2_train_scaled)
reshaped_X3_train_scaled = reshape_data(X3_train_scaled)

In [23]:
# Reshape testing data
reshaped_X1_test_scaled = reshape_data(X1_test_scaled)
reshaped_X2_test_scaled = reshape_data(X2_test_scaled)
reshaped_X3_test_scaled = reshape_data(X3_test_scaled)

In [24]:
# Reshape X_full
reshaped_X1_full = reshape_data(X1_full_scaled)
reshaped_X2_full = reshape_data(X2_full_scaled)
reshaped_X3_full = reshape_data(X3_full_scaled)

## Build Model

In [25]:
# Initialize model
model = Sequential()

In [26]:
# Add layers
model.add(LSTM(128, input_shape=(reshaped_X1_train_scaled.shape[1],1), return_sequences=True))
model.add(Dropout(0.4))
model.add(BatchNormalization())  # Normalize activation outputs

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(LSTM(128))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(2, activation='softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [27]:
# Compile model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

**Train and predict on X1-Y1 data (recession 1 quarter out)**

In [28]:
# Fit the model to the training data
# Shuffle True/False to randomize the training data rows being fed into the model
model.fit(reshaped_X1_train_scaled, y1_train, validation_split=0.33, epochs=100, shuffle=True, verbose=2)

Train on 91 samples, validate on 45 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
91/91 - 10s - loss: 1.0315 - acc: 0.5165 - val_loss: 0.6813 - val_acc: 0.7111
Epoch 2/100
91/91 - 1s - loss: 0.8932 - acc: 0.6044 - val_loss: 0.6708 - val_acc: 0.7111
Epoch 3/100
91/91 - 1s - loss: 0.8664 - acc: 0.6484 - val_loss: 0.6591 - val_acc: 0.7111
Epoch 4/100
91/91 - 1s - loss: 0.5326 - acc: 0.7143 - val_loss: 0.6492 - val_acc: 0.7111
Epoch 5/100
91/91 - 1s - loss: 0.5818 - acc: 0.7582 - val_loss: 0.6408 - val_acc: 0.7111
Epoch 6/100
91/91 - 1s - loss: 0.5473 - acc: 0.6923 - val_loss: 0.6339 - val_acc: 0.7111
Epoch 7/100
91/91 - 1s - loss: 0.4270 - acc: 0.7912 - val_loss: 0.6294 - val_acc: 0.7111
Epoch 8/100
91/91 - 1s - loss: 0.3731 - acc: 0.8352 - val_loss: 0.6273 - val_acc: 0.7111
Epoch 9/100
91/91 - 1s - loss: 0.3905 - acc: 0.8242 - val_loss: 0.6259 - val_acc: 0.7111
Epoch 10/100
91/91 - 1s - loss: 0.3315 - acc: 0.8571 - val_l

Epoch 88/100
91/91 - 1s - loss: 0.1280 - acc: 0.9451 - val_loss: 1.8735 - val_acc: 0.7111
Epoch 89/100
91/91 - 1s - loss: 0.0789 - acc: 0.9670 - val_loss: 1.8881 - val_acc: 0.7111
Epoch 90/100
91/91 - 1s - loss: 0.0697 - acc: 0.9890 - val_loss: 1.9119 - val_acc: 0.7111
Epoch 91/100
91/91 - 2s - loss: 0.2008 - acc: 0.9341 - val_loss: 1.9681 - val_acc: 0.7111
Epoch 92/100
91/91 - 1s - loss: 0.1093 - acc: 0.9451 - val_loss: 2.0025 - val_acc: 0.7111
Epoch 93/100
91/91 - 1s - loss: 0.1797 - acc: 0.9341 - val_loss: 2.0231 - val_acc: 0.7111
Epoch 94/100
91/91 - 1s - loss: 0.0701 - acc: 1.0000 - val_loss: 2.0311 - val_acc: 0.7111
Epoch 95/100
91/91 - 1s - loss: 0.0833 - acc: 0.9670 - val_loss: 2.0471 - val_acc: 0.7111
Epoch 96/100
91/91 - 1s - loss: 0.1264 - acc: 0.9451 - val_loss: 2.0412 - val_acc: 0.7111
Epoch 97/100
91/91 - 1s - loss: 0.0773 - acc: 0.9670 - val_loss: 2.0230 - val_acc: 0.7111
Epoch 98/100
91/91 - 1s - loss: 0.0839 - acc: 0.9670 - val_loss: 1.9933 - val_acc: 0.7111
Epoch 99/1

In [29]:
# Validate model using test data
model_loss1, model_accuracy1 = model.evaluate(reshaped_X1_test_scaled, y1_test, verbose=2)

34/34 - 0s - loss: 1.1776 - acc: 0.8529


In [30]:
# Make predictions using test data
predictions1 = model.predict_classes(reshaped_X1_test_scaled)

In [31]:
# Compare results
one_qtr_out = pd.DataFrame({"Predicted":predictions1, "Actual":y1_test})
one_qtr_out

,Predicted,Actual
quarter,,
2006Q1,0,0.0
1988Q2,0,0.0
2012Q1,0,0.0
2015Q1,0,0.0
2009Q3,0,0.0
1981Q2,0,1.0
2006Q4,0,0.0
1993Q3,0,0.0
1987Q1,0,0.0


In [32]:
# Save model
name1 = f"shuffled-1q-out-{int(time.time())}"
model.save(f"models/{name1}.h5")

#### Predict on full X1

In [49]:
# pred_X1_full = model.predict_classes(reshaped_X1_full)
# X1_full_results = pd.DataFrame({"Predicted":pred_X1_full, "Actual":y1})
# X1_full_results.loc[X1_full_results["Actual"]==1]
X1_full_results.to_csv("resources/X1_full_results.csv")

**Train and predict on X2-Y2 data (recession 2 quarters out)**

In [35]:
# Fit the model to the training data
model.fit(reshaped_X2_train_scaled, y2_train, validation_split=0.33, epochs=100, shuffle=True, verbose=2)

Train on 90 samples, validate on 45 samples
Epoch 1/100
90/90 - 1s - loss: 0.4189 - acc: 0.9444 - val_loss: 1.9346 - val_acc: 0.7333
Epoch 2/100
90/90 - 1s - loss: 0.5253 - acc: 0.9222 - val_loss: 1.8718 - val_acc: 0.7333
Epoch 3/100
90/90 - 1s - loss: 0.2849 - acc: 0.9111 - val_loss: 1.8186 - val_acc: 0.7333
Epoch 4/100
90/90 - 1s - loss: 0.2090 - acc: 0.9444 - val_loss: 1.7762 - val_acc: 0.7333
Epoch 5/100
90/90 - 1s - loss: 0.1719 - acc: 0.9556 - val_loss: 1.7458 - val_acc: 0.7333
Epoch 6/100
90/90 - 1s - loss: 0.2154 - acc: 0.9444 - val_loss: 1.7237 - val_acc: 0.7333
Epoch 7/100
90/90 - 1s - loss: 0.2750 - acc: 0.9111 - val_loss: 1.6927 - val_acc: 0.7333
Epoch 8/100
90/90 - 1s - loss: 0.1992 - acc: 0.9444 - val_loss: 1.6519 - val_acc: 0.7333
Epoch 9/100
90/90 - 1s - loss: 0.1784 - acc: 0.9222 - val_loss: 1.6145 - val_acc: 0.7333
Epoch 10/100
90/90 - 1s - loss: 0.1848 - acc: 0.9444 - val_loss: 1.5742 - val_acc: 0.7333
Epoch 11/100
90/90 - 1s - loss: 0.2079 - acc: 0.9222 - val_loss: 

Epoch 92/100
90/90 - 1s - loss: 0.0183 - acc: 1.0000 - val_loss: 2.0800 - val_acc: 0.7333
Epoch 93/100
90/90 - 1s - loss: 0.0387 - acc: 0.9889 - val_loss: 2.0830 - val_acc: 0.7333
Epoch 94/100
90/90 - 1s - loss: 0.0181 - acc: 1.0000 - val_loss: 2.0578 - val_acc: 0.7333
Epoch 95/100
90/90 - 1s - loss: 0.0542 - acc: 0.9889 - val_loss: 1.9894 - val_acc: 0.7333
Epoch 96/100
90/90 - 1s - loss: 0.0151 - acc: 1.0000 - val_loss: 1.8841 - val_acc: 0.7333
Epoch 97/100
90/90 - 1s - loss: 0.0555 - acc: 0.9556 - val_loss: 1.8552 - val_acc: 0.7333
Epoch 98/100
90/90 - 1s - loss: 0.0131 - acc: 1.0000 - val_loss: 1.8564 - val_acc: 0.7333
Epoch 99/100
90/90 - 1s - loss: 0.0149 - acc: 1.0000 - val_loss: 1.8565 - val_acc: 0.7333
Epoch 100/100
90/90 - 1s - loss: 0.0654 - acc: 0.9778 - val_loss: 1.9118 - val_acc: 0.7111


In [36]:
# Validate model using test data
model_loss2, model_accuracy2 = model.evaluate(reshaped_X2_test_scaled, y2_test, verbose=2)

34/34 - 0s - loss: 1.4436 - acc: 0.7647


In [37]:
# Make predictions using test data
predictions2 = model.predict_classes(reshaped_X2_test_scaled)

In [38]:
# Compare results
two_qtrs_out = pd.DataFrame({"Predicted":predictions2, "Actual":y2_test})
two_qtrs_out

,Predicted,Actual
quarter,,
2006Q1,0,0.0
2015Q1,0,0.0
2012Q1,0,0.0
1996Q2,0,0.0
2009Q3,1,0.0
1981Q1,0,1.0
2006Q4,0,0.0
1976Q4,0,0.0
1999Q2,1,0.0


In [39]:
# Save model
name2 = f"shuffled-2q-out-{int(time.time())}"
model.save(f"models/{name2}.h5")

#### Predict on full X2

In [50]:
# pred_X2_full = model.predict_classes(reshaped_X2_full)
# X2_full_results = pd.DataFrame({"Predicted":pred_X2_full, "Actual":y2})
# X2_full_results.loc[X2_full_results["Actual"]==1]
X2_full_results.to_csv("resources/X2_full_results.csv")

**Train and predict on X3-Y3 data (recession 4 quarters out)**

In [41]:
# Fit the model to the training data
model.fit(reshaped_X3_train_scaled, y3_train, validation_split=0.33, epochs=100, shuffle=True, verbose=2)

Train on 89 samples, validate on 44 samples
Epoch 1/100
89/89 - 1s - loss: 0.9361 - acc: 0.7865 - val_loss: 1.2406 - val_acc: 0.7955
Epoch 2/100
89/89 - 1s - loss: 0.3674 - acc: 0.8764 - val_loss: 1.1338 - val_acc: 0.7500
Epoch 3/100
89/89 - 1s - loss: 0.6178 - acc: 0.8652 - val_loss: 1.0798 - val_acc: 0.7500
Epoch 4/100
89/89 - 1s - loss: 0.2582 - acc: 0.9101 - val_loss: 0.9527 - val_acc: 0.7727
Epoch 5/100
89/89 - 1s - loss: 0.1889 - acc: 0.9438 - val_loss: 0.8934 - val_acc: 0.7955
Epoch 6/100
89/89 - 1s - loss: 0.2964 - acc: 0.9101 - val_loss: 0.8168 - val_acc: 0.7727
Epoch 7/100
89/89 - 1s - loss: 0.2500 - acc: 0.8652 - val_loss: 0.8135 - val_acc: 0.7273
Epoch 8/100
89/89 - 1s - loss: 0.1980 - acc: 0.9326 - val_loss: 0.7587 - val_acc: 0.7273
Epoch 9/100
89/89 - 1s - loss: 0.1816 - acc: 0.9101 - val_loss: 0.6721 - val_acc: 0.7500
Epoch 10/100
89/89 - 1s - loss: 0.2466 - acc: 0.8539 - val_loss: 0.6349 - val_acc: 0.7727
Epoch 11/100
89/89 - 1s - loss: 0.2128 - acc: 0.8652 - val_loss: 

Epoch 92/100
89/89 - 1s - loss: 0.0555 - acc: 0.9888 - val_loss: 1.1618 - val_acc: 0.8182
Epoch 93/100
89/89 - 1s - loss: 0.0778 - acc: 0.9551 - val_loss: 1.1380 - val_acc: 0.8182
Epoch 94/100
89/89 - 1s - loss: 0.0333 - acc: 0.9888 - val_loss: 1.1936 - val_acc: 0.7955
Epoch 95/100
89/89 - 1s - loss: 0.0614 - acc: 0.9663 - val_loss: 1.2005 - val_acc: 0.7955
Epoch 96/100
89/89 - 1s - loss: 0.0374 - acc: 0.9888 - val_loss: 1.1978 - val_acc: 0.7955
Epoch 97/100
89/89 - 1s - loss: 0.0354 - acc: 0.9888 - val_loss: 1.1971 - val_acc: 0.7955
Epoch 98/100
89/89 - 1s - loss: 0.0747 - acc: 0.9663 - val_loss: 1.1523 - val_acc: 0.8182
Epoch 99/100
89/89 - 1s - loss: 0.0317 - acc: 0.9888 - val_loss: 1.1669 - val_acc: 0.8409
Epoch 100/100
89/89 - 1s - loss: 0.1020 - acc: 0.9663 - val_loss: 1.2014 - val_acc: 0.8409


In [42]:
# Validate model using test data
model_loss3, model_accuracy3 = model.evaluate(reshaped_X3_test_scaled, y3_test, verbose=2)

34/34 - 0s - loss: 0.3332 - acc: 0.8824


In [43]:
# Make predictions using test data
predictions3 = model.predict_classes(reshaped_X3_test_scaled)

In [44]:
# Compare results
four_qtrs_out = pd.DataFrame({"Predicted":predictions3, "Actual":y3_test})
four_qtrs_out

,Predicted,Actual
quarter,,
2007Q3,0,1.0
1976Q4,0,0.0
2018Q1,0,0.0
2009Q3,0,0.0
1996Q2,0,0.0
2015Q4,0,0.0
1977Q1,0,0.0
1993Q3,0,0.0
2009Q4,0,0.0


In [45]:
# Save model
name3 = f"shuffled-4q-out-{int(time.time())}"
model.save(f"models/{name3}.h5")

#### Predict on full X3

In [51]:
# pred_X3_full = model.predict_classes(reshaped_X3_full)
# X3_full_results = pd.DataFrame({"Predicted":pred_X3_full, "Actual":y3})
# X3_full_results.loc[X3_full_results["Actual"]==1]
X3_full_results.to_csv("resources/X3_full_results.csv")